### In this notebook we try to test different initial conditions for the optimization algorithms. Be careful : this notebook take a lot of time to run entirely (≈ 30 min)

In [8]:
cd(@__DIR__)
include("../utils/utils.jl")

MAPE (generic function with 2 methods)

### Initial conditions

In [9]:
Φ_init=[-100,-1,0,1e-5,1,5,10,100,1000]
σ_init=[1e-10,1e-5,0.1,1,5,50,100,1000,10000]

9-element Vector{Float64}:
     1.0e-10
     1.0e-5
     0.1
     1.0
     5.0
    50.0
   100.0
  1000.0
 10000.0

Making a calendar of 100 years

In [10]:
try 
    using Dates
catch ; 
    import Pkg
    Pkg.add("Dates")
    using Dates
end
k=100
DatesRange=collect(Date(1900):Day(1):Date(1900+k))
N=length(DatesRange)

36525

## Simulating a series following an AR(1) model

In [11]:
x0=7.
Φ,σ=0.7,4.

(0.7, 4.0)

Simulating the series

In [12]:
include("../utils/Simulation.jl")
x=[x0 ; simulation(x0,Φ,σ,N-1)]

36525-element Vector{Float64}:
  7.0
 -0.7823190074480939
  0.5870647132724636
  6.491947434579019
  0.6042852651022028
 -4.80090942589429
  2.9613300363414994
  7.108664014642528
 14.085565594811808
  7.299133790710543
  ⋮
  3.637946699214281
  1.3440343736573401
  3.6691469956990863
  0.574364733094592
  7.007298626327332
  6.187947824575876
  8.605697421566306
  8.1479638422418
  6.017144581922489

# Testing the initial parameters with Nelder-Mead algorithm

In [13]:
include("../utils/Estimation.jl")
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-10,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062
2,1.0e-5,0.70062,0.70062,0.70062,0.700621,0.70062,0.70062,0.70062,0.70062,0.70062
3,0.1,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.700621,0.70062,0.70062
4,1.0,0.70062,0.70062,0.70062,0.70062,0.70062,0.700621,0.70062,0.70062,0.70062
5,5.0,0.70062,0.70062,0.700621,0.70062,0.70062,0.700621,0.70062,0.70062,0.70062
6,50.0,0.70062,0.70062,-0.00148471,-0.001475,1.00242,0.92271,-0.260421,0.70062,-0.0652259
7,100.0,-1.24699e151,2.26143,-0.00648818,-0.00647947,2.99907,2.10621,1.85803,-1.24824e151,1.70116
8,1000.0,-1.41951e151,-2.91732,0.0295357,0.0295516,3.31741,-0.0795364,1.73058,-56.5993,1.77601
9,10000.0,45.3612,-2.21945,-0.0100691,-0.0100612,3.52591,17.1484,5.07261,10.9584,1.42312e151


We notice that estimated Φ values change completely for σ_init² ≥ 50

In [14]:
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init, sigma_est=true)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-10,4.01911,4.01912,4.01911,4.01911,4.01911,4.01911,4.01911,4.01912,4.01911
2,1.0e-5,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
3,0.1,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
4,1.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
5,5.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
6,50.0,4.01911,4.01912,1.41082e-67,1.41082e-67,2.53725e-62,1.18023e-61,3.27572e-66,4.01911,1.91784e-63
7,100.0,6.75263e75,1.63075e-61,5.05901e-69,5.05901e-69,1.77568e-68,7.63619e-66,2.4393e-66,5.75462e75,2.5542e-65
8,1000.0,2.65372e76,4.75802e-68,8.11865e-68,8.11865e-68,4.54997e-67,2.51923e-68,3.22061e-66,5.83513e-64,7.57586e-64
9,10000.0,3.49503e-66,1.71888e-65,3.25766e-64,3.25766e-64,7.57024e-66,7.57024e-66,2.29561e-63,1.5377e-64,3.46875e76


For σ_init² ≥ 50, estimated σ values are aberrant. 
However, for both Φ and σ estimations, the values remain constant as long as σ_init² < 50 (maybe there is a critical value).
Φ_init doesn't seem to have an impact on the result, except for σ_init = 50 where for some Φ_init the estimation didn't become unstable.
Below we try to find this treshold of instability.

In [15]:
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init, σ, sigma_est=true)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-10,0.477822,0.477911,0.477814,0.477867,0.477836,0.477861,0.477842,0.47788,0.477831
2,1.0e-5,0.477874,0.477836,0.477863,0.477819,0.477847,0.477871,0.477841,0.477867,0.477831
3,0.1,0.477852,0.477856,0.47785,0.477847,0.477823,0.477826,0.47787,0.477851,0.477842
4,1.0,0.477837,0.477853,0.477839,0.477828,0.477818,0.477822,0.477834,0.477824,0.477854
5,5.0,0.477836,0.477796,0.477872,0.477841,0.477835,0.477849,0.47781,0.477834,0.477835
6,50.0,0.477848,0.477879,100.0,100.0,100.0,100.0,100.0,0.477806,100.0
7,100.0,1.68816e77,100.0,100.0,100.0,100.0,100.0,100.0,1.43865e77,100.0
8,1000.0,6.6343e77,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
9,10000.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,8.67188e77


In [16]:
σ_init2=collect(5:10:50)
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init2)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,5,0.70062,0.70062,0.700621,0.70062,0.70062,0.700621,0.70062,0.70062,0.70062
2,15,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062
3,25,0.340408,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.338718,0.70062
4,35,0.70062,-1.18468,0.357947,0.358029,2.77912,0.700618,0.460887,0.70062,0.70062
5,45,0.70062,-1.66323,-0.0115803,-0.0115726,3.30801,0.983588,-0.341121,0.70062,-0.129442


In [17]:
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init2, sigma_est=true)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,5.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
2,15.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01912,4.01911
3,25.0,2.49124e-64,4.01911,4.01912,4.01911,4.01911,4.01911,4.01911,1.45233e-60,4.01911
4,35.0,4.01911,2.66145e-59,8.1225e-63,8.1225e-63,5.6856e-66,4.01911,6.98134e-62,4.01911,4.01912
5,45.0,4.01911,3.6596e-65,6.41548e-69,6.41548e-69,1.21853e-61,1.37648e-62,6.66259e-65,4.01911,2.65892e-63


Unstable values appear for σ_init² ≥ 15. 
Let's try a finer search : 

In [18]:
σ_init3=collect(5:1:15)
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init3)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,5,0.70062,0.70062,0.700621,0.70062,0.70062,0.700621,0.70062,0.70062,0.70062
2,6,0.70062,0.70062,0.700621,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062
3,7,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062
4,8,0.70062,0.700621,0.70062,0.70062,0.70062,0.70062,0.700621,0.70062,0.70062
5,9,0.70062,0.700621,0.732795,0.732951,0.70062,0.70062,0.70062,0.700621,0.70062
6,10,0.70062,0.700621,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062
7,11,0.70062,0.70062,0.700621,0.70062,0.70062,0.700621,0.70062,0.70062,0.70062
8,12,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.70062,0.700621
9,13,0.70062,0.70062,0.70062,0.70062,0.700621,0.70062,0.70062,0.700621,0.70062


In [19]:
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init3, sigma_est=true)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,5.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
2,6.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01912,4.01911,4.01911,4.01911
3,7.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01912
4,8.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
5,9.0,4.01911,4.01911,2.14078e-62,2.14078e-62,4.01911,4.01911,4.01911,4.01911,4.01911
6,10.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
7,11.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
8,12.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911
9,13.0,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911,4.01911


Unstable values appear for σ_init² ≥ 9

For stable values, let's see if the initial conditions have an impact on the accuracy of estimation : 

In [20]:
σ_init4=[1e-5 ; collect(1:2:7)]
println("MAPE table (percentage) :")
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init4, Φ)

MAPE table (percentage) :


Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-5,0.0885718,0.0886136,0.0885482,0.0886572,0.0885838,0.0885835,0.0886245,0.0886012,0.0886068
2,1.0,0.0885898,0.0885811,0.0886285,0.0886062,0.0885728,0.0886496,0.088642,0.088551,0.0886025
3,3.0,0.0886944,0.0885162,0.0885761,0.0886512,0.0885544,0.0886344,0.0885984,0.0885846,0.0886089
4,5.0,0.0885992,0.0886356,0.0886724,0.0885825,0.0885604,0.0886499,0.088596,0.0886128,0.0885968
5,7.0,0.0885704,0.0886055,0.0886269,0.0885924,0.0886052,0.0886216,0.0885931,0.0885844,0.0886194


In [21]:
println("MAPE table (percentage) :")
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init4, sigma_est=true, σ)

MAPE table (percentage) :


Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-5,0.477874,0.477836,0.477863,0.477819,0.477847,0.477871,0.477841,0.477867,0.477831
2,1.0,0.477837,0.477853,0.477839,0.477828,0.477818,0.477822,0.477834,0.477824,0.477854
3,3.0,0.477857,0.477829,0.477826,0.477896,0.47789,0.477858,0.477799,0.477834,0.477822
4,5.0,0.477836,0.477796,0.477872,0.477841,0.477835,0.477849,0.47781,0.477834,0.477835
5,7.0,0.477829,0.477834,0.477806,0.477836,0.477836,0.477812,0.477811,0.477811,0.477879


# AR model on each month

Parameters for each month

In [22]:
Φ_month=[4,1,8,2,7,9,2,4,1,3,1,2]/10
σ_month=[1.5,2,4.5,7,8,3,4.5,1,7,2.5,3,6]

12-element Vector{Float64}:
 1.5
 2.0
 4.5
 7.0
 8.0
 3.0
 4.5
 1.0
 7.0
 2.5
 3.0
 6.0

Generating the series associated

In [23]:
n_month=200
x=SimulateMonth(4.,Date(2000,11,1),Φ_month,σ_month,n_month)
Date_vec=collect(Date(2000,11,1):(Date(2000,11,1) + Month(n_month) - Day(1)))

6086-element Vector{Date}:
 2000-11-01
 2000-11-02
 2000-11-03
 2000-11-04
 2000-11-05
 2000-11-06
 2000-11-07
 2000-11-08
 2000-11-09
 2000-11-10
 ⋮
 2017-06-22
 2017-06-23
 2017-06-24
 2017-06-25
 2017-06-26
 2017-06-27
 2017-06-28
 2017-06-29
 2017-06-30

Separation of the months

In [24]:
include(raw"..\utils\Separation.jl")
Monthly_date=MonthlySeparateDates(Date_vec)
Monthly_temp=MonthlySeparateX(x,Date_vec)

12-element Vector{Vector{Any}}:
 [[-0.8742944132189718, 1.5380226629411402, -1.374460472739487, -2.087856965374933, -2.0533668399300287, -0.3068925752715106, -2.7521921183823985, -0.8070469337985233, -1.0736089890502973, -3.557841118578305  …  1.1798558912780837, 0.2131769888633976, -1.1706064304341077, 2.1506065138016894, 0.9420866218324002, -2.0753514051652653, -2.7180033262916705, -0.4868430263627238, -1.3479349432416734, -0.1515956203150729], [-2.163280511528164, -0.44066377383437827, 0.07107063258357277, -0.7661473587961299, -0.5242290002702481, 0.17265018710402233, 0.4861793776198813, -2.802644771992067, -2.5827357603200074, -1.1475143579981693  …  0.023626879836710024, -0.969215304976802, 2.277163491670322, 2.6587509715164703, 0.9231820005671857, 2.103756383497975, 0.8747137172400139, 1.0621660512911657, -0.9865727591690163, 2.1553435464274284], [2.5723743086615523, 1.9136173690672296, 2.6013373734188514, 0.6280794670128926, -0.7129397360724437, -1.692263802710891, -1.4174342833

## 1.One model per year and per month

In [25]:
MeanMonthlyEstimation(x,p,Estimators::Vector=[zeros(p) ; 1e-5],algo=NelderMead())=
(Monthly_stats(mean,MonthlyEstimation(Monthly_temp,p,Estimators,algo),1),
Monthly_stats(mean,MonthlyEstimation(Monthly_temp,p,Estimators,algo),2))

MeanMonthlyEstimation (generic function with 3 methods)

In [26]:
Monthly_Estimators=MonthlyEstimation(Monthly_temp,1)

12-element Vector{Vector{Any}}:
 [(0.2571643972039014, 1.4942433066529743), (0.37501804629518415, 1.342344408594641), (0.5910713222795357, 1.786164666840303), (0.22258981318501508, 1.3708262651331538), (0.30246733134360404, 1.8779603168347554), (0.2456979518602695, 1.677098374990132), (0.7153270129070535, 1.425627059921448), (0.31227809368739995, 1.2103642806258743), (0.020850391804022185, 1.167688007758888), (0.3029030390696757, 1.5343280738923628), (0.2971385936885589, 1.6167261382937015), (0.22437131257734988, 1.4721712311695823), (0.551295125681446, 1.5069044656486708), (0.2654473489085167, 1.8815730909741657), (0.2643617357666759, 1.4926247528064558), (0.5191549764367509, 1.3871174587591), (0.48256957203823503, 1.3358743107654707)]
 [(0.12083231527990776, 2.281281350143772), (0.33122847541278144, 1.8930852751977527), (-0.10542438282834422, 1.831469125550127), (0.27890434024196187, 1.880164510656696), (0.17386300708029978, 2.052935173060575), (0.3361169730854379, 1.912345877290476)

In [27]:
Monthly_stats(mean,Monthly_Estimators,1)

12-element Vector{Any}:
 0.34998270969018785
 0.11382110485983733
 0.7551940487106338
 0.2587444494422747
 0.6248129617495133
 0.8707422119469506
 0.14510022135366493
 0.4082198128748544
 0.06314900240449156
 0.3695891136303688
 0.014578604015184016
 0.20262471425406822

In [28]:
MeanMonthlyEstimation(Monthly_Estimators,1)

(Any[0.34998270969018785, 0.11382110485983733, 0.7551940487106338, 0.2587444494422747, 0.6248129617495133, 0.8707422119469506, 0.14510022135366493, 0.4082198128748544, 0.06314900240449156, 0.3695891136303688, 0.014578604015184016, 0.20262471425406822], Any[1.5046844829212753, 1.8872691189541058, 4.657554458902432, 6.885282199523715, 7.859424214436346, 2.875521226420322, 4.3708110201400086, 0.9964028817837451, 6.276522439825367, 2.360667561140532, 3.025664280104385, 5.514798750187526])

In [29]:
σ_init=[1e-5,1,3,5,7,9,15,50]

8-element Vector{Float64}:
  1.0e-5
  1.0
  3.0
  5.0
  7.0
  9.0
 15.0
 50.0

In [30]:
GridEstimation(MeanMonthlyEstimation, Monthly_temp , Φ_init, σ_init)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…
1,1.0e-5,"Any[0.349984, 0.11382, 0.755191, 0.258744, 0.624815, 0.870742, 0.145091, 0.408219, 0.0631486, 0.369586, 0.0145751, 0.202621]","Any[0.349985, 0.113818, 0.755194, 0.258746, 0.624811, 0.870742, 0.145092, 0.408222, 0.0631452, 0.369581, 0.0145681, 0.202621]","Any[0.349983, 0.113821, 0.755194, 0.258744, 0.624813, 0.870742, 0.1451, 0.40822, 0.063149, 0.369589, 0.0145786, 0.202625]","Any[0.349985, 0.113812, 0.755197, 0.258748, 0.624812, 0.870743, 0.145095, 0.408223, 0.0631493, 0.369584, 0.0145728, 0.202616]","Any[0.349981, 0.113824, 0.755194, 0.258744, 0.624816, 0.870743, 0.145085, 0.40822, 0.0631496, 0.369588, 0.0145747, 0.202619]","Any[0.349984, 0.11382, 0.755195, 0.258748, 0.624812, 0.870742, 0.145104, 0.408221, 0.0631476, 0.369585, 0.0145712, 0.202618]","Any[0.349984, 0.113823, 0.755194, 0.258743, 0.624812, 0.87074, 0.145093, 0.408218, 0.063151, 0.369584, 0.0145675, 0.202616]","Any[0.349984, 0.113826, 0.755194, 0.258745, 0.624814, 0.870743, 0.145099, 0.408224, 0.06315, 0.369583, 0.0145674, 0.202623]","Any[0.349985, 0.113819, 0.755195, 0.258748, 0.624815, 0.870743, 0.145095, 0.408229, 0.0631477, 0.369588, 0.0145731, 0.202624]"
2,1.0,"Any[0.349988, 0.113828, 0.755194, 0.258746, 0.624814, 0.870744, 0.145096, 0.408224, 0.0631468, 0.369586, 0.0145708, 0.202621]","Any[0.349982, 0.113822, 0.755193, 0.258747, 0.624816, 0.870742, 0.145097, 0.408219, 0.0631456, 0.36959, 0.0145647, 0.202621]","Any[0.349981, 0.113821, 0.751523, 0.258749, 0.624815, 0.964793, 0.145096, 0.40822, 0.0631448, 0.36959, 0.0145743, 0.202622]","Any[0.349986, 0.113824, 0.751555, 0.258744, 0.624814, 0.964946, 0.145099, 0.408222, 0.0631465, 0.369587, 0.0145751, 0.202616]","Any[0.349985, 0.113826, 0.755192, 0.258749, 0.624814, 0.870743, 0.145091, 0.408225, 0.0631502, 0.369585, 0.0145774, 0.202625]","Any[0.349984, 0.113823, 0.755192, 0.258744, 0.624811, 0.870745, 0.145098, 0.408224, 0.0631445, 0.369583, 0.0145698, 0.20262]","Any[0.349977, 0.11382, 0.755195, 0.25875, 0.624811, 0.870742, 0.145098, 0.408221, 0.0631506, 0.369586, 0.0145761, 0.202623]","Any[0.349977, 0.113823, 0.755193, 0.258745, 0.624813, 0.870743, 0.145093, 0.40822, 0.0631432, 0.369588, 0.0145691, 0.202622]","Any[0.34998, 0.113818, 0.755193, 0.258747, 0.624812, 0.870744, 0.145097, 0.384062, 0.0631465, 0.369586, 0.0145768, 0.202618]"
3,3.0,"Any[0.322648, 0.113823, 0.755197, 0.258748, 0.624814, 0.870739, 0.145097, 2.337, 0.0631488, 0.369583, 0.0145723, 0.202614]","Any[0.494368, 0.113819, 0.755195, 0.258748, 0.624812, 0.870743, 0.145098, 0.230084, 0.0631448, 0.369583, 0.014574, 0.202621]","Any[0.30162, 0.113822, 0.790062, 0.258747, 0.628136, 0.878156, 0.145097, 0.00187091, 0.0631443, 0.369587, 0.0145709, 0.202622]","Any[0.301624, 0.113821, 0.790153, 0.258744, 0.628151, 0.878312, 0.145098, 0.00188129, 0.063146, 0.369589, 0.0145759, 0.202624]","Any[0.353195, 0.11382, 0.755192, 0.258748, 0.624808, 0.870744, 0.145093, 0.314013, 0.063141, 0.369587, 0.0145705, 0.202618]","Any[0.630632, 0.113819, 0.755196, 0.258745, 0.624813, 0.870742, 0.145097, 0.802391, 0.0631475, 0.369584, 0.0145712, 0.202621]","Any[0.515064, 0.113815, 0.755195, 0.258742, 0.624814, 0.870743, 0.145097, 0.0536507, 0.0631455, 0.369589, 0.0145766, 0.202617]","Any[0.302089, 0.113821, 0.755193, 0.258749, 0.624812, 0.870746, 0.145093, 2.61158, 0.0631475, 0.36958, 0.0145688, 0.202621]","Any[0.536675, 0.11382, 0.755195, 0.258742, 0.624815, 0.871171, 0.145094, -0.671066, 0.0631423, 0.369584, 0.0145736, 0.20262]"
4,5.0,"Any[1.09711, 0.113819, 0.755194, 0.25875, 0.624816, 0.870741, 0.145095, 0.873308, 0.0631489, 0.336261, 0.0145715, 0.202615]","Any[0.415238, 0.0674634, 0.755194, 0.258746, 0.624811, 0.870741, 0.145103, 0.196461, 0.0631443, 0.196824, 0.014576, 0.202621]","Any[0.0985082, 0.139815, 0.728307, 0.258749, 0.656638, 1.0525, 0.145091, -0.026964, 0.0631434, 0.354567, 0.0145702, 0.202617]","Any[0.0985185

Reminder : 
```julia
Φ_month=[4,1,8,2,7,9,2,4,1,3,1,2]/10
σ_month=[1.5,2,4.5,7,8,3,4.5,1,7,2.5,3,6]
```

In [31]:
println("MAPE table (percentage) :")
GridEstimation(MeanMonthlyEstimation, Monthly_temp , Φ_init, σ_init, Φ_month)

MAPE table (percentage) :


Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-5,20.965,20.9657,20.9647,20.9642,20.9655,20.965,20.9653,20.9658,20.9655
2,1.0,20.966,20.9663,21.3329,21.3334,20.9657,20.9657,20.965,20.9665,21.1255
3,3.0,61.7173,25.2578,29.6251,29.6215,22.5194,32.9399,29.3638,67.8668,44.9094
4,5.0,43.2097,26.8023,37.7201,37.721,30.7882,154.521,41.052,28.1213,69.4714
5,7.0,98.8435,81.5873,44.971,44.97,91.9152,3.47502e153,142.807,53.7252,79.8186
6,9.0,5.21364e153,178.145,46.8584,46.8573,141.311,317.353,173.64,70.1463,91.0801
7,15.0,2.07946e154,115.911,46.2079,46.2088,174.602,327.015,157.59,1.6852e153,1.54048e154
8,50.0,4.69193e154,593.053,60.4521,60.4445,488.231,329.532,3.65017e153,1.88002e153,1.67777e154


Like before, we see that the more σ_init is close to 0 the better is the estimation.

In [32]:
println("MAPE table (percentage) :")
GridEstimation(MeanMonthlyEstimation, Monthly_temp , Φ_init, σ_init, σ_month, sigma_est=true)

MAPE table (percentage) :


Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-5,3.75649,3.75645,3.7564,3.75646,3.75649,3.75644,3.75645,3.75651,3.7564
2,1.0,3.75639,3.75652,10.308,10.3081,3.75643,3.75643,3.75659,3.75639,4.19611
3,3.0,12.6873,11.842,23.9474,23.9475,12.3894,8.54234,9.54791,12.6873,9.96013
4,5.0,15.9034,17.2576,31.511,31.5111,16.4791,16.0353,13.7561,15.9032,19.1512
5,7.0,22.4361,20.64,39.9732,39.9732,23.0805,1.05215e76,23.3127,22.4361,22.883
6,9.0,1.84937e76,27.8045,43.1265,43.1266,27.3457,27.9313,26.3926,29.4287,28.0313
7,15.0,1.10763e77,37.8253,51.9383,51.9383,36.7391,34.9659,32.8938,7.65555e75,6.84333e76
8,50.0,2.70337e77,65.7651,74.2125,74.2125,64.4186,61.0078,5.63877e76,7.89792e75,9.45619e76


It is the same for estimating σ.

## 2.Each month concatanated with all years

In [33]:
σ_init=[1e-5,1,3,5,7,9,15,50]
GridEstimation(MonthlyConcatanatedEstimation, Monthly_temp, Φ_init, σ_init)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…
1,1.0e-5,"Any[0.347149, 0.111278, 0.777778, 0.271414, 0.626188, 0.871447, 0.153408, 0.422376, 0.0518056, 0.390611, 0.0161472, 0.209365]","Any[0.347145, 0.111274, 0.777776, 0.271416, 0.62618, 0.871447, 0.153405, 0.422381, 0.051814, 0.390612, 0.016145, 0.209364]","Any[0.347154, 0.11127, 0.777779, 0.271417, 0.626185, 0.871447, 0.153407, 0.42238, 0.0518092, 0.390606, 0.0161565, 0.209366]","Any[0.347156, 0.111277, 0.777776, 0.271418, 0.626185, 0.871447, 0.153407, 0.422393, 0.0518149, 0.390613, 0.0161491, 0.20937]","Any[0.347153, 0.111269, 0.77778, 0.271413, 0.626182, 0.871447, 0.153406, 0.42238, 0.0518135, 0.390612, 0.016144, 0.209368]","Any[0.347155, 0.111275, 0.777777, 0.27142, 0.626183, 0.871448, 0.15341, 0.422374, 0.0518123, 0.390607, 0.0161443, 0.209364]","Any[0.347157, 0.111264, 0.777781, 0.271416, 0.626182, 0.871448, 0.153403, 0.422377, 0.0518098, 0.390606, 0.0161547, 0.209366]","Any[0.34716, 0.111273, 0.777779, 0.271418, 0.626185, 0.871448, 0.153404, 0.422373, 0.0518063, 0.390614, 0.0161478, 0.209367]","Any[0.347153, 0.11127, 0.777779, 0.271417, 0.626186, 0.871447, 0.153406, 0.422378, 0.0518177, 0.390606, 0.0161464, 0.209363]"
2,1.0,"Any[0.347153, 0.111276, 0.777778, 0.271417, 0.626184, 0.87145, 0.153406, 0.422375, 0.0518133, 0.390611, 0.0161462, 0.209362]","Any[0.347159, 0.111271, 0.77778, 0.271412, 0.626183, 0.871448, 0.153404, 0.422377, 0.0518136, 0.39061, 0.0161512, 0.209367]","Any[0.347157, 0.111269, 0.777777, 0.271417, 0.626184, 0.871448, 0.153405, 0.42238, 0.0518062, 0.390612, 0.0161412, 0.209367]","Any[0.347156, 0.111271, 0.777779, 0.271416, 0.626184, 0.955374, 0.153413, 0.422374, 0.0518169, 0.390606, 0.0161516, 0.209365]","Any[0.347159, 0.111273, 0.777777, 0.271418, 0.626185, 0.871454, 0.153405, 0.422372, 0.0518094, 0.39061, 0.0161475, 0.20937]","Any[0.347163, 0.111271, 0.777778, 0.271414, 0.626185, 0.87145, 0.153409, 0.422377, 0.0518061, 0.390609, 0.0161411, 0.209363]","Any[0.347156, 0.111273, 0.777778, 0.271412, 0.626187, 0.871449, 0.153405, 0.422378, 0.0518122, 0.390614, 0.0161498, 0.209366]","Any[0.347155, 0.111273, 0.777776, 0.271418, 0.626187, 0.871447, 0.153405, 0.422375, 0.0518116, 0.390606, 0.0161476, 0.209368]","Any[0.347156, 0.11128, 0.77778, 0.271414, 0.626185, 0.871448, 0.153406, 0.42238, 0.0518107, 0.390609, 0.0161468, 0.209368]"
3,3.0,"Any[0.347157, 0.11127, 0.777774, 0.271415, 0.626186, 0.871449, 0.153406, 3.12479, 0.0518074, 0.390611, 0.0161465, 0.209364]","Any[0.347158, 0.111273, 0.777779, 0.271411, 0.626183, 0.871444, 0.153407, 0.190385, 0.051814, 0.390615, 0.0161483, 0.209366]","Any[0.347159, 0.111269, 1.42453, 0.271415, 0.626183, 0.709157, 0.153404, 0.00469525, 0.0518105, 0.390613, 0.0161451, 0.209374]","Any[0.347156, 0.111272, 1.42483, 0.271412, 0.626186, 0.709309, 0.153406, 0.00470619, 0.05181, 0.390609, 0.0161458, 0.209362]","Any[0.347158, 0.111272, 0.777775, 0.271414, 0.626183, 0.871447, 0.153402, -0.242008, 0.0518121, 0.390611, 0.0161422, 0.20936]","Any[0.347156, 0.111275, 0.777775, 0.271413, 0.626188, 0.87145, 0.153411, 0.42238, 0.0518094, 0.390613, 0.0161512, 0.209367]","Any[0.347155, 0.111271, 0.77778, 0.271414, 0.626187, 0.871445, 0.153405, 0.422377, 0.0518117, 0.390608, 0.0161509, 0.209365]","Any[0.347159, 0.111276, 0.777776, 0.27142, 0.626184, 0.871447, 0.153406, 2.25632, 0.0518162, 0.39061, 0.0161462, 0.209365]","Any[0.347155, 0.111274, 0.777778, 0.271422, 0.62619, 0.871447, 0.153407, 0.422375, 0.0518143, 0.390614, 0.0161491, 0.209372]"
4,5.0,"Any[0.34715, 0.111272, 0.777776, 0.271415, 0.626183, 0.871448, 0.153407, 1.96906, 0.0518106, 0.39061, 0.0161473, 0.209368]","Any[0.347161, -0.716933, 0.777778, 0.271415, 0.626182, 0.87145, 0.153404, 3.41078, 0.0518125, 0.390608, 0.016149, 0.209365]","Any[-0.0274192, 0.11127, 0.256422, 0.271415, 0.62618, 0.256422, 0.153405, -0.0274192, 0.051817, 0.390611, 0.0161457, 0.209364]","Any[-0.0274147, 0.11127

Values start to fluctuate from σ_init² = 5 for monthly estimation.

In [34]:
σ_init=[1e-5,1,2,3,4,5]
println("MAPE table (percentage) :")
GridEstimation(MonthlyConcatanatedEstimation, Monthly_temp, Φ_init, σ_init, Φ_month)

MAPE table (percentage) :


Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-5,22.7096,22.7093,22.7081,22.7093,22.7088,22.709,22.7076,22.7094,22.708
2,1.0,22.7089,22.7081,22.7097,22.9557,22.7091,22.7091,22.7086,22.7089,22.7095
3,2.0,22.708,22.7087,36.4735,36.4761,22.7087,22.7088,22.7082,22.7085,22.7088
4,3.0,79.0092,26.6092,38.2553,38.2562,35.618,22.7087,22.7082,60.916,22.709
5,4.0,51.6385,27.548,31.7082,31.7099,76.562,55.3012,59.7337,51.8873,144.338
6,5.0,54.9316,152.105,50.0758,50.0751,97.6348,363.89,166.197,46.3402,52.9893


Values start to fluctuate from σ_init² = 2 for monthly estimation. It seems that in every case the initial value for σ must be close to 0 for best results.

In [35]:
σ_init=[1e-5,1,2,3,4,5]
println("MAPE table (percentage) :")
GridEstimation(MonthlyConcatanatedEstimation, Monthly_temp, Φ_init, σ_init, σ_month, sigma_est=true)

MAPE table (percentage) :


Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-5,5.87085,5.87078,5.87082,5.87084,5.87085,5.87079,5.87078,5.87072,5.8708
2,1.0,5.87074,5.87082,5.87088,12.268,5.87083,5.87076,5.87077,5.8708,5.87086
3,2.0,5.87074,5.87075,20.434,20.434,5.87075,5.87072,5.87086,5.87078,5.87085
4,3.0,14.0368,14.0368,28.1766,28.1766,14.0369,5.87074,5.87076,14.0369,5.87084
5,4.0,14.0369,14.0369,20.434,20.434,14.0368,14.0369,14.0369,14.0368,14.0368
6,5.0,14.0369,22.1122,35.9602,35.9602,21.8204,14.0368,14.0368,14.0368,14.0369


When we estimate σ, values became also unsatisfactory from σ_init² = 2.

## 3.One likelihood for each month

In [36]:
σ_init=[1e-5,1,3,5,7,9,15,50]
GridEstimation(MonthlyEstimationSumLL, Monthly_temp, Φ_init, σ_init)

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…,Array…
1,1.0e-5,"Any[0.384404, 0.124502, 0.792455, 0.268797, 0.661727, 0.903752, 0.154551, 0.428835, 0.0604397, 0.397066, 0.0197908, 0.220694]","Any[0.384403, 0.124492, 0.792453, 0.268812, 0.6625, 0.90375, 0.154548, 0.428836, 0.0604499, 0.397065, 0.0197872, 0.220696]","Any[0.384407, 0.124497, 0.792453, 0.26881, 0.661725, 0.903753, 0.15456, 0.428836, 0.060449, 0.397068, 0.019786, 0.22069]","Any[0.384403, 0.1245, 0.792452, 0.26881, 0.661725, 0.90375, 0.154556, 0.428829, 0.0604386, 0.397063, 0.0197896, 0.220696]","Any[0.384404, 0.124499, 0.792449, 0.268809, 0.661723, 0.903752, 0.154555, 0.428846, 0.0604377, 0.397071, 0.0197906, 0.220697]","Any[0.384404, 0.124502, 0.792452, 0.268809, 0.661724, 0.903752, 0.154554, 0.428834, 0.0604422, 0.397064, 0.019792, 0.220691]","Any[0.384408, 0.124499, 0.792453, 0.268809, 0.661722, 0.903753, 0.154552, 0.428839, 0.0604422, 0.397067, 0.0197901, 0.220693]","Any[0.384404, 0.124493, 0.792453, 0.26881, 0.661723, 0.903752, 0.154554, 0.428834, 0.0604477, 0.397065, 0.0197892, 0.220696]","Any[0.384405, 0.1245, 0.79245, 0.268809, 0.661725, 0.90375, 0.154549, 0.428838, 0.0604429, 0.397061, 0.0197955, 0.220694]"
2,1.0,"Any[0.384403, 0.124501, 0.792454, 0.268809, 0.661727, 0.903752, 0.154551, 0.428838, 0.0604441, 0.397063, 0.0197887, 0.220694]","Any[0.3844, 0.1245, 0.792451, 0.268812, 0.661724, 0.903751, 0.154555, 0.428836, 0.0604435, 0.397066, 0.0197902, 0.220693]","Any[0.384402, 0.124503, 0.792453, 0.268809, 0.661722, 0.747595, 0.154548, 0.428839, 0.0604446, 0.397065, 0.0197944, 0.220693]","Any[0.384403, 0.124498, 0.792455, 0.268806, 0.661724, 0.747755, 0.154555, 0.428838, 0.0604366, 0.397069, 0.0197924, 0.220696]","Any[0.384402, 0.124492, 0.792455, 0.268802, 0.661723, 0.903751, 0.154551, 0.428833, 0.0604463, 0.397067, 0.0197944, 0.220695]","Any[0.384402, 0.124503, 0.792452, 0.268805, 0.661721, 0.903752, 0.154556, 0.428838, 0.0604409, 0.397068, 0.01979, 0.220696]","Any[0.384405, 0.124501, 0.792451, 0.26881, 0.661723, 0.903752, 0.154551, 0.428836, 0.0604441, 0.397067, 0.019796, 0.220696]","Any[0.384402, 0.124499, 0.792454, 0.268809, 0.661725, 0.903751, 0.15455, 0.428836, 0.060444, 0.397066, 0.0197924, 0.220692]","Any[0.384406, 0.1245, 0.792452, 0.268811, 0.661723, 0.903751, 0.154559, 0.428836, 0.0604406, 0.397062, 0.0197853, 0.2207]"
3,3.0,"Any[0.384403, 0.124501, 0.792451, 0.268808, 0.661726, 0.903752, 0.154548, 3.12479, 0.0604393, 0.397067, 0.0197948, 0.220694]","Any[0.384404, 0.1245, 0.79245, 0.268814, 0.661724, 0.903754, 0.154556, 0.190385, 0.0604474, 0.397061, 0.0197901, 0.220694]","Any[0.384403, 0.1245, 1.42453, 0.26881, 0.661726, 0.709157, 0.154552, 0.00469525, 0.0604457, 0.397068, 0.0197945, 0.220698]","Any[0.384393, 0.1245, 1.42483, 0.268815, 0.661725, 0.709309, 0.154551, 0.00470619, 0.06044, 0.397063, 0.0197925, 0.220697]","Any[0.384405, 0.124495, 0.792455, 0.268814, 0.661721, 0.90375, 0.154548, 0.901606, 0.0604447, 0.39707, 0.0197873, 0.220698]","Any[0.531812, 0.124503, 0.792454, 0.268815, 0.661725, 0.903752, 0.154552, 0.428837, 0.0604505, 0.397066, 0.0197964, 0.220696]","Any[-0.0711885, 0.124496, 0.792453, 0.268811, 0.661727, 0.903752, 0.154552, 0.428838, 0.0604397, 0.39707, 0.0197899, 0.220693]","Any[0.384403, 0.124501, 0.792453, 0.268813, 0.661723, 0.903751, 0.15455, 2.25632, 0.06044, 0.397067, 0.0197952, 0.220692]","Any[0.384398, 0.124499, 0.792451, 0.268809, 0.661722, 0.903752, 0.154554, 0.428839, 0.0604428, 0.397069, 0.0197881, 0.220694]"
4,5.0,"Any[0.384401, 0.124499, 0.792455, 0.268803, 0.661724, 0.90375, 0.154553, 1.96906, 0.0604456, 0.397066, 0.01979, 0.220695]","Any[0.384409, 0.124505, 0.792453, 0.268801, 0.661726, 0.903752, 0.154548, 3.41078, 0.0604398, 0.397065, 0.0197955, 0.220696]","Any[-0.0274192, 0.124498, 0.256422, 0.268806, 0.661721, 0.878262, 0.154551, -0.0274192, 0.0604428, 0.397071, 0.0197908, 0.220691]","Any[-0.0274147, 0.124503, 0.25648

Values start to fluctuate from σ_init² = 2 for monthly estimation with sum of likelihood.

In [37]:
σ_init=[1e-5,1,2,3,4,5]
println("MAPE table (percentage) :")
GridEstimation(MonthlyEstimationSumLL, Monthly_temp, Φ_init, σ_init, Φ_month)

MAPE table (percentage) :


Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-5,21.836,21.8263,21.8353,21.8363,21.8369,21.8359,21.8362,21.8352,21.8358
2,1.0,21.8363,21.8362,23.2126,23.2113,21.8346,21.8365,21.8359,21.8358,21.8367
3,2.0,21.8363,21.8359,33.3417,33.34,21.837,29.0121,21.8353,21.8362,21.8361
4,3.0,78.002,25.6019,37.6299,37.6322,31.6858,24.2568,31.3278,59.9089,21.8366
5,4.0,50.631,26.5403,38.5639,38.5632,75.5543,54.2932,58.7255,50.8794,143.331
6,5.0,53.9237,83.9598,44.4696,44.4684,88.4242,371.972,177.655,45.3335,51.9821


Here, even from σ_init² = 1 estimations start to be less accurate.

In [38]:
σ_init=[1e-5,1,2,3,4,5]
println("MAPE table (percentage) :")
GridEstimation(MonthlyEstimationSumLL, Monthly_temp, Φ_init, σ_init, σ_month, sigma_est=true)

MAPE table (percentage) :


Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-5,3.20889,7.7596,3.20866,3.20884,3.2088,3.2088,3.20878,3.20887,3.20878
2,1.0,3.20883,3.2088,11.3491,11.3491,3.20875,3.20879,3.20879,3.2088,3.2088
3,2.0,3.20893,3.20885,19.6087,19.6087,3.20868,11.4685,3.20879,3.20882,3.20893
4,3.0,11.4685,11.4684,27.4749,27.475,11.4684,11.2984,11.2986,11.4683,3.20896
5,4.0,11.4685,11.4686,27.6986,27.6986,11.4683,11.4682,11.4684,11.4683,11.4684
6,5.0,11.4684,11.4684,35.5648,35.5647,11.4684,19.5581,19.5581,11.4684,11.4684


Just like before, only for σ_init² ≈ 0 estimated parameters do not become very inaccurate.

## Conclusion

Φ_init does not seem to have an significant impact on the quality of estimation of Φ and σ. However, if σ_init is superior or equal to 1, some parameters estimated became very innacurate and the greater σ_init is, the more parameters tend to explode. 

# Testing the initial parameters with BFGS algorithm

In [39]:
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init[1:3],algo=BFGS())

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-5,-1.39355,0.301968,0.283648,0.269054,0.712288,0.698532,0.375854,1.40625,-34.1445
2,1.0,-167.283,0.517915,0.517915,0.517915,0.517915,0.517915,0.495636,329.185,0.290587
3,2.0,124.635,0.517915,0.517915,0.517915,0.517915,0.517915,0.469706,34.8514,0.456701


In [40]:
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init[4:5],algo=BFGS())

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,3.0,48.4654,0.517915,0.517915,0.517915,0.517915,0.517915,0.517915,0.585714,0.504816
2,4.0,-0.200559,0.517915,0.517915,0.517915,0.517915,0.517915,0.517915,-43.1316,0.505275


In [41]:
GridEstimation(LL_AR_Estimation,x, Φ_init, [σ_init[6]],algo=BFGS())

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,5.0,-98.0109,0.517915,0.517915,0.517915,0.517915,-15.0202,0.517915,-8.89883,0.515286


Here the BFGS algorithm can be very slow but more importantly the output is far from the true value and change significantly from one initial Φ to another. 

# Testing the initial parameters with Gradient Descent algorithm

In [ ]:
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init[1:3],algo=GradientDescent())

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0e-10,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
2,1.0e-5,25.2559,0.53981,0.301765,0.39506,0.770449,0.660339,1.8396,18.0371,-103.141
3,0.1,0.696702,0.694819,0.683459,0.65634,0.711496,0.693351,0.678675,-2.74501e5,-2.41922


In [ ]:
GridEstimation(LL_AR_Estimation,x, Φ_init, σ_init[4:5],algo=GradientDescent())

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,-410.869,0.696632,0.696756,0.696756,0.697798,-1356.42,-25031.4,0.697254,-45349.1
2,5.0,2.15766e6,0.6968,0.696792,0.696792,0.698144,0.697885,0.697222,-6.08279e6,1.0915e7


In [ ]:
GridEstimation(LL_AR_Estimation,x, Φ_init, [σ_init[6]],algo=GradientDescent())

Row,σ_init² ↓ | Φ_init →,-100.0,-1.0,0.0,1.0e-5,1.0,5.0,10.0,100.0,1000.0
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,50.0,96555.3,0.689629,0.0,1.0e-5,1.0,0.697585,0.69965,0.679309,0.676631


The results are very similar to the results given by the BFGS algorithm : a lot of them are inaccurate and the algorithm is very slow. The Nelder-Mead algorithm seems to be the best optimization algorithm for our problem.